In [1]:
%load_ext autoreload

In [2]:
import os
import sys
import re
from argparse import Namespace

sys.path.append('../src-py/')

In [3]:
%autoreload

from utils import *
from project_debater_api import *

In [4]:
import torch
import json

import nltk
import numpy as np
import pandas as pd

from pathlib import Path
from datasets import load_dataset, load_metric, Dataset

from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import BartTokenizer, BartForConditionalGeneration

In [5]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [6]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [7]:
ceph_dir = '/home/sile2804/data-ceph/arguana/arg-generation/multi-taks-counter-argument-generation'
local_home_dir = '../data'

In [8]:
#Evalute ROUGE scores
gen_kwargs = {
    "do_sample": True, 
    "max_length":200,
    "top_p":0.95, 
    "top_k":50,
}

def generate_ds_attacks(ds, model, tokenizer, premises_clm, conclusion_clm, gen_kwargs, skip_special_tokens=True, batch_size=8):
    ds = ds.map(lambda x :preprocess_function(x, tokenizer, premises_clm, 'counter', conclusion_clm=conclusion_clm), batched=True)
    ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
    dataloader = torch.utils.data.DataLoader(ds, batch_size=batch_size)
    attacks = generate_counters(model, tokenizer, dataloader, gen_kwargs, skip_special_tokens=skip_special_tokens)
    
    return attacks

#### Load Testing data:

##### Loading validation data from Reddit:

In [9]:
valid_df = pd.read_pickle(ceph_dir + '/reddit_data/conclusion_and_ca_generation/valid_conclusion_comp_remove_75sem_perc_with_targets.pkl')

In [10]:
#drop duplicate posts
unique_valid_posts = valid_df.drop_duplicates('post_id')

In [11]:
valid_ds = Dataset.from_pandas(unique_valid_posts)
tmp_ds = valid_ds.train_test_split(0.9)
sample_valid_ds = tmp_ds['test']

#saving the sample
reddit_sample_valid_ds = sample_valid_ds.flatten_indices()
reddit_sample_valid_ds.save_to_disk('../data/sample_valid_ds')

  0%|          | 0/3 [00:00<?, ?ba/s]

In [13]:
#loading the random sample of validation dataset with the auto-generated conclusion
reddit_sample_valid_ds = Dataset.load_from_disk('../data/sample_valid_ds/')

##### Loading validation data from Kialo:

In [29]:
valid_ds = Dataset.from_pandas(pd.read_pickle(ceph_dir + '/kialo_data/kialo_valid_df.pkl'))

In [10]:
tmp_ds = valid_ds.train_test_split(0.5)
sample_valid_ds = tmp_ds['test']

#saving the sample
sample_valid_ds = sample_valid_ds.flatten_indices()
sample_valid_ds.save_to_disk('../data/kialo_sample_valid_ds')

  0%|          | 0/3 [00:00<?, ?ba/s]

In [9]:
sample_valid_ds = Dataset.load_from_disk('../data/kialo_sample_valid_ds')

### Evaluating on Kialo:

In [ ]:
#KIALO models
masked_conclusion_kialo_model = BartForConditionalGeneration.from_pretrained(local_home_dir + '/output/masked-conclusion-kialo-model/').to(device)
masked_conclusion_kialo_tokenizer = BartTokenizer.from_pretrained(local_home_dir + '/output/masked-conclusion-kialo-model/')
known_conclusion_kialo_model = BartForConditionalGeneration.from_pretrained(local_home_dir + '/output/known-conclusion-kialo-model/').to(device)
known_conclusion_kialo_tokenizer = BartTokenizer.from_pretrained(local_home_dir + '/output/known-conclusion-kialo-model/')

#masked_conclusion_kialo_model = BartForConditionalGeneration.from_pretrained(local_home_dir + '/output/masked-conclusion-kialo-unique-model/').to(device)
#known_conclusion_kialo_model = BartForConditionalGeneration.from_pretrained(local_home_dir + '/output/known-conclusion-kialo-unique-model/').to(device)

In [11]:
known_conc_attacks = generate_ds_attacks(sample_valid_ds, known_conclusion_kialo_model, known_conclusion_kialo_tokenizer, 'premises', 'conclusion_text', gen_kwargs)
masked_conc_attacks = generate_ds_attacks(sample_valid_ds, masked_conclusion_kialo_model, masked_conclusion_kialo_tokenizer, 'premises', None, gen_kwargs)
#auto_conc_attacks = generate_ds_attacks(sample_valid_ds, known_conclusion_model, tokenizer, 'masked_premises', 'auto_conclusion', gen_kwargs)

  0%|          | 0/3 [00:00<?, ?ba/s]

/usr/local/lib/python3.6/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


  0%|          | 0/3 [00:00<?, ?ba/s]

In [12]:
kialo_pred_df = pd.DataFrame(list(zip(sample_valid_ds['conclusion_text'], sample_valid_ds['premises'], 
                                   sample_valid_ds['counter'], known_conc_attacks, masked_conc_attacks)), 
                      columns=['conclusion', 'premises', 'gt', 'known_conc_attacks', 'masked_conc_attacks'])

kialo_pred_df['argument'] = kialo_pred_df.apply(lambda row: row['conclusion'] + ' : ' + ' '.join(row['premises']), axis=1)
kialo_pred_df['premises'] = kialo_pred_df['premises'].apply(lambda x: ' '.join(x))

In [13]:
kialo_pred_df[['argument', 'gt', 'known_conc_attacks', 'masked_conc_attacks']].sample(10).head(n=10)

,argument,gt,known_conc_attacks,masked_conc_attacks
404,"In some German states it is forbidden to host dance events and concerts on certain holidays. : This leads to a decrease in happiness and leisure activities for non-Christians. These bans have existed in Germany since at least the early 20th century Dance events and concerts contribute to the economy as food and beverages are generally consumed in large quantities, and tickets normally being required for attendance.","The German dance scene is similar to dance scenes in other countries where it is allowed to host dance events on Easter. Thus, no major economic effect is visible.","Some German states have banned cultural events and concerts on certain holidays, such as Christmas Eve and New Year's Eve.","Dance events and concerts contribute to the economy as food and beverages are generally consumed in large quantities, and tickets normally being required for attendance."
4,"When one person is having sex only to get paid, that sex is not fully consensual and so should not be allowed or encouraged. : The sex industry is often a means of providing for oneself or a family when all other options have been exhausted. Choice is always contingent on circumstances; the most free choices are made when all options are equally viable. For sex workers, choosing not to have sex is not a viable choice because it directly undermines their livelihood. When many sex workers have a history of abuse, you cannot assume that consent can be given. Monetary exchange is an inequitable power dynamic that undermines free choice. We should not permit sexual activity that does not proceed from personal relationships that exist on a level field.","Under this logic, there are a series of jobs that should simply be banished, like mining, for instance: it's something miners are doing only to get paid, so taking action on the job is not fully consensual, and would thus configure slavery, and should not be allowed or encouraged.\nThe same could be said of countless jobs.","The sex industry is often a means of providing for oneself or a family when all other options have been exhausted. Choice is always contingent on circumstances; the most free choices are made when all options are equally viable. For sex workers, choosing not to have sex is not a viable choice because it directly undermines their livelihood.","The sex industry is often a means of providing for oneself or a family when all other options have been exhausted. Choice is always contingent on circumstances; the most free choices are made when all options are equally viable. For sex workers, choosing not to have sex is not a viable choice because it directly undermines their livelihood. When many sex workers have a history of abuse, you cannot assume that consent can be given. Monetary exchange is an inequitable power dynamic that undermines free choice."
615,"Judge by Intent : Friends gained in by judging their intent will want to be better with you Intention allows for failure in action that may not always be possible; if a friend truly cares for you, their sentiment is more important than their mistakes. The action is observable, so the judging actually occurs when we examine intent. Intent is hard to establish, because we rely on asking the offender to explain why they did what they did. We use clues, such as intonation of voice, expression of face, choice that led to action, previous actions etc, to determine the motivation and to judge the character and intent. The action is observable and a fact of occurence, not needing judgement.",Friends gained by intent may fail at certain actions causing complications to both lives,"Intention is hard to establish, because we rely on asking the offender to explain why they did what they did. We use clues, such as intonation of voice, expression of face, choice that led to action, previous actions etc, to determine the motivation and to judge the character and intent. The action is observable and a fac

### Generate predictions on Reddit:

In [14]:
# Loading Reddit models

#masked_conclusion_tokenizer = BartTokenizer.from_pretrained(local_home_dir + '/output/extended_tokenizer')
#masked_conclusion_model = BartForConditionalGeneration.from_pretrained(local_home_dir + '/output/masked-conclusion-bart-model/').to(device)

known_conclusion_model = BartForConditionalGeneration.from_pretrained(local_home_dir  + '/output/known-conclusion-bart-model').to(device)
known_conclusion_tokenizer = BartTokenizer.from_pretrained(local_home_dir + '/output/extended_tokenizer')

pred_conclusion_model = BartForConditionalGeneration.from_pretrained(local_home_dir  + '/output/pred-conclusion-bart-model/').to(device)
pred_conclusion_tokenizer = BartTokenizer.from_pretrained(local_home_dir + '/output/pred-conclusion-bart-model/')

In [15]:
# Generate attacks for different settings
known_conc_attacks  = generate_ds_attacks(reddit_sample_valid_ds, known_conclusion_model, known_conclusion_tokenizer, 'masked_premises', 'title', gen_kwargs)
bart_conc_attacks   = generate_ds_attacks(reddit_sample_valid_ds, known_conclusion_model, known_conclusion_tokenizer, 'masked_premises', 'bart_conclusion', gen_kwargs)
masked_conc_attacks = generate_ds_attacks(reddit_sample_valid_ds, known_conclusion_model, known_conclusion_tokenizer, 'masked_premises', None, gen_kwargs)
#This model would predict the conclusion and then the counter in one sequence.
joint_conc_attacks  = generate_ds_attacks(reddit_sample_valid_ds, pred_conclusion_model, pred_conclusion_tokenizer, 'masked_premises', None, gen_kwargs, skip_special_tokens=False)

  0%|          | 0/3 [00:00<?, ?ba/s]

/usr/local/lib/python3.6/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [16]:
reddit_pred_df = pd.DataFrame(list(zip(
                                       reddit_sample_valid_ds['post_id'],
                                       reddit_sample_valid_ds['title'], 
                                       reddit_sample_valid_ds['conclusion_targets'],
                                       reddit_sample_valid_ds['conclusion_stance'],
                                       reddit_sample_valid_ds['bart_conclusion'], 
                                       reddit_sample_valid_ds['masked_premises'], 
                                       reddit_sample_valid_ds['counter'], 
                                       known_conc_attacks, masked_conc_attacks, 
                                       bart_conc_attacks, joint_conc_attacks)), 
                columns=['post_id', 'conclusion', 'conclusion_target', 'conclusion_stance', 'bart_conclusion', 'premises', 'gt_attack', 'known_conc_attacks', 
                         'masked_conc_attacks', 'bart_conc_attacks', 'joint_conc_attacks'])

reddit_pred_df['argument'] = reddit_pred_df.apply(lambda row: row['conclusion'] + ' : ' + ' '.join(row['premises']), axis=1)
reddit_pred_df['premises'] = reddit_pred_df['premises'].apply(lambda x: ' '.join(x))

#process the jointly generated conclusion and counter
reddit_pred_df['joint_conc'] = reddit_pred_df['joint_conc_attacks'].apply (lambda x: x.split('<counter>')[0])
reddit_pred_df['joint_conc_attacks'] = reddit_pred_df['joint_conc_attacks'].apply (lambda x: x.split('<counter>')[1] if '<counter>' in x else x)
reddit_pred_df['joint_conc'] = reddit_pred_df['joint_conc'].apply (lambda x: re.sub('<s>|</s>|<conclusion>|<counter>|<pad>', '', x).strip())
reddit_pred_df['joint_conc_attacks'] = reddit_pred_df['joint_conc_attacks'].apply (lambda x: re.sub('<s>|</s>|<conclusion>|<counter>|<pad>', '', x).strip())

#save created pred_df
reddit_pred_df.to_pickle('../data/output/reddit_pred_df.pkl')

In [17]:
reddit_pred_df[['argument', 'conclusion' , 'joint_conc', 'masked_conc_attacks', 'known_conc_attacks', 'joint_conc_attacks']].sample(10).head(n=10)

,argument,conclusion,joint_conc,masked_conc_attacks,known_conc_attacks,joint_conc_attacks
110,"I am in favor of the establishment of quasi-national sports teams for indigenous groups within countries. : below are my two top points in favour the establishment of 'quasi national' sports teams. i look forward to your responses! 1. native american tribes rarely give their consent to have their image and culture used yet teams such as the cleveland indians frequently go ahead and used mascots inspired from indian culture regardless of if it will be deemed offensive or not. however, if quasi national sports teams are formed for indigenous groups they will then be able to reclaim their own culture. this effect, will also likely get those who are in marginalized indigenous groups to feel empowered and consequently cultivate an interest in sports. why is this important? because it would help them stay away from gangs, drugs, and other crime. 2. indigenous people were colonized. they had their own distinct culture and heritage prior to being imperialized. therefore, by giving indigenous people their own sports team they regain some long lost cultural individuality. also, it clearly would be way more fun for an indigenous person to root for a team dedicated specifically designed for them. by analogy, if a british guy is a fan of football he is likely not going to want to root for germany in the world cup if he can root for england instead. why? because germany has it's own distinct culture and history, and brits would rather root for something they relate too, i. e their country and culture.",I am in favor of the establishment of quasi-national sports teams for indigenous groups within countries.,I believe that there should be quasi-national sports teams for indigenous groups.,i don't think there is much of a difference between a 'quasi national' team and a 'national' team.,i don't think it's fair to say that native americans aren't interested in sports teams. it's not like they don't want to be involved in them. they're not interested in being involved in the culture of the native american people. they want to get involved in their country and culture.,"i think the biggest problem with this idea is that it's not about the rights of indigenous people, it's about the right of the people to own their own sports team. i think it's important to understand that the idea of a quasi national sports team is a form of cultural appropriation."
1385,"Children ought never to be put on psychiatric medication. : my main argument is this: children, physiologically speaking, have not reached maturity. duh. no one disagrees with this. their brains, bodies, emotions and basically everything about them is not mature. a child can display both acceptable and unacceptable behavior at any given time. when the behavior is deemed unacceptable and persistently so, this is when psychiatrists are called upon. this is bad news for the child. since children have not reached puberty, no less their formative years, their enduring personality along with their body, brain and character have only just begun to develop into what in the future can become a stable, integrated self. studies have shown the potential of stultification of children, stunting their growth after prolonged usage of adhd related drugs. interfering chemically with a brain not yet matured must then be seen as having the potential for tremendous, far reaching negative consequences. the idea i am challenging is believing a child and their bad or aberrant behavior necessarily equates to a brain based biological chemical imbalance and or disease process needing the treatment of a psychiatrist who will administer powerful brain and mind altering psychotropic chemicals to a still developing brain. that psychiatric medications certainly alter brain chemistry in anyone taking them is indisputable. premature psychiatric action upon developing children inherently stifles and impedes the natural course of growth 

### Evaluate generated predictions:

In [33]:
reddit_pred_df = pd.read_pickle('../data/output/reddit_pred_df.pkl')

In [34]:
#collect references
arg_counters = valid_df.groupby('post_id').agg({
    'counter': lambda x: [' '.join(c) for c in x]
}).reset_index()
arg_counters = pd.Series(arg_counters.counter.values, index=arg_counters.post_id).to_dict()

reddit_pred_df['gt_attack'] = reddit_pred_df['gt_attack'].apply(lambda x: str(x))
reddit_pred_df['all_counters'] = reddit_pred_df['post_id'].apply(lambda x: arg_counters[x])

In [36]:
reddit_pred_df = reddit_pred_df[reddit_pred_df.all_counters.map(len) > 0]

In [47]:
masked_conc_eval = evaluate_gen_attacks(reddit_pred_df['masked_conc_attacks'].tolist(), reddit_pred_df['all_counters'].tolist())
known_conc_eval = evaluate_gen_attacks(reddit_pred_df['known_conc_attacks'].tolist(), reddit_pred_df['all_counters'].tolist())
bart_conc_eval = evaluate_gen_attacks(reddit_pred_df['bart_conc_attacks'].tolist(), reddit_pred_df['all_counters'].tolist())
pred_conc_eval = evaluate_gen_attacks(reddit_pred_df['joint_conc_attacks'].tolist(), reddit_pred_df['all_counters'].tolist())

##### Test stance correcteness:

In [21]:
filtered_reddit_pred_df = reddit_pred_df[pd.notna(reddit_pred_df.conclusion_target)]

In [22]:
#compute the stance of the generated counters towards the conclusion target
filtered_reddit_pred_df['masked_conc_stances'] = get_stances(filtered_reddit_pred_df.conclusion_target.tolist(), filtered_reddit_pred_df.masked_conc_attacks.tolist())
filtered_reddit_pred_df['known_conc_stances']  = get_stances(filtered_reddit_pred_df.conclusion_target.tolist(), filtered_reddit_pred_df.known_conc_attacks.tolist())
filtered_reddit_pred_df['bart_conc_stances']   = get_stances(filtered_reddit_pred_df.conclusion_target.tolist(), filtered_reddit_pred_df.bart_conc_attacks.tolist())
filtered_reddit_pred_df['joint_conc_stances']  = get_stances(filtered_reddit_pred_df.conclusion_target.tolist(), filtered_reddit_pred_df.joint_conc_attacks.tolist())

ProConClient: 100%|██████████| 2172/2172 [00:37<00:00, 54.74it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
ProConClient: 100%|██████████| 2172/2172 [00:37<00:00, 59.12it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until

ProConClient: 100%|██████████| 2172/2172 [01:15<00:00, 28.62it/s]

ProConClient:

In [23]:
#compute the distance between the conclusion stance and the attack stance: the bigger the distance the better...
masked_conc_stance_score1 = round(np.mean([abs(x[0] - x[1]) for x in zip(filtered_reddit_pred_df.masked_conc_stances.tolist(), filtered_reddit_pred_df.conclusion_stance.tolist())]), 2)
known_conc_stance_score1 = round(np.mean([abs(x[0] - x[1]) for x in zip(filtered_reddit_pred_df.known_conc_stances.tolist(), filtered_reddit_pred_df.conclusion_stance.tolist())]), 2)
bart_conc_stance_score1 = round(np.mean([abs(x[0] - x[1]) for x in zip(filtered_reddit_pred_df.bart_conc_stances.tolist(), filtered_reddit_pred_df.conclusion_stance.tolist())]), 2)
joint_conc_stance_score1 = round(np.mean([abs(x[0] - x[1]) for x in zip(filtered_reddit_pred_df.joint_conc_stances.tolist(), filtered_reddit_pred_df.conclusion_stance.tolist())]), 2)

#check if the two stances are contradicotry
masked_conc_stance_score2 = round(sum([int(x[0] * x[1] < 0) for x in zip(filtered_reddit_pred_df.masked_conc_stances.tolist(), filtered_reddit_pred_df.conclusion_stance.tolist())])/len(filtered_reddit_pred_df), 2)
known_conc_stance_score2  = round(sum([int(x[0] * x[1] < 0) for x in zip(filtered_reddit_pred_df.known_conc_stances.tolist(), filtered_reddit_pred_df.conclusion_stance.tolist())])/len(filtered_reddit_pred_df), 2)
bart_conc_stance_score2   = round(sum([int(x[0] * x[1] < 0) for x in zip(filtered_reddit_pred_df.bart_conc_stances.tolist(), filtered_reddit_pred_df.conclusion_stance.tolist())])/len(filtered_reddit_pred_df), 2)
joint_conc_stance_score2  = round(sum([int(x[0] * x[1] < 0) for x in zip(filtered_reddit_pred_df.joint_conc_stances.tolist(), filtered_reddit_pred_df.conclusion_stance.tolist())])/len(filtered_reddit_pred_df), 2)

In [24]:
from tabulate import tabulate

print(tabulate([['Masked Conclusion'] + [round(masked_conc_eval['bleu'], 2), masked_conc_eval['bert-fscore'], masked_conc_stance_score1, masked_conc_stance_score2],
                ['BART Conclusion'] + [round(bart_conc_eval['bleu'], 2), bart_conc_eval['bert-fscore'], bart_conc_stance_score1, bart_conc_stance_score2],
                ['Joint Prediction'] + [round(pred_conc_eval['bleu'], 2), pred_conc_eval['bert-fscore'], joint_conc_stance_score1, joint_conc_stance_score2],
                ['Known Conclusion'] + [round(known_conc_eval['bleu'], 2), known_conc_eval['bert-fscore'], known_conc_stance_score1, known_conc_stance_score2],
    ], headers=['bleu', 'bert-f1score', 'stance-score (diff)', 'stance-score (agreement)']))

                     bleu    bert-f1score    stance-score (diff)    stance-score (agreement)
-----------------  ------  --------------  ---------------------  --------------------------
Masked Conclusion    0.16            0.14                   0.82                        0.55
BART Conclusion      0.18            0.16                   0.85                        0.55
Joint Prediction     0.17            0.16                   0.83                        0.55
Known Conclusion     0.19            0.16                   0.85                        0.55


ProConClient: 100%|██████████| 2172/2172 [00:52<00:00, 57.11it/s]

##### Similarity to conclusion:

In [15]:
masked_conc_eval = evaluate_gen_attacks(reddit_pred_df['conclusion'], reddit_pred_df['masked_conc_attacks'].tolist())
known_conc_eval  = evaluate_gen_attacks(reddit_pred_df['conclusion'], reddit_pred_df['known_conc_attacks'].tolist())
#auto_conc_eval   = evaluate_gen_attacks(reddit_pred_df['conclusion'], reddit_pred_df['auto_conc_attacks'].tolist())
pred_conc_eval   = evaluate_gen_attacks(reddit_pred_df['conclusion'], reddit_pred_df['pred_conc_attacks'].tolist())
gt_attack_conc_eval = evaluate_gen_attacks(reddit_pred_df['conclusion'], reddit_pred_df['gt'].tolist())

In [17]:
from tabulate import tabulate

print(tabulate([['masked'] + [round(masked_conc_eval['bleu'], 2), masked_conc_eval['bert-fscore']],
                #['auto'] + list(auto_conc_eval.values()),
                ['pred'] + [round(pred_conc_eval['bleu'], 2), pred_conc_eval['bert-fscore']],
                ['known'] + [round(known_conc_eval['bleu'], 2), known_conc_eval['bert-fscore']],
                ['gt_attack'] + [round(gt_attack_conc_eval['bleu'], 2), gt_attack_conc_eval['bert-fscore']]
    ], headers=['bleu', 'bert-f1score']))

             bleu    bert-f1score
---------  ------  --------------
masked       0               0.16
pred         0.01            0.18
known        0.03            0.33
gt_attack    0              -0.03
